In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
from read_bases import *
from scipy import constants

from cpfX import hum1zpf16m #for Humlıcek approximation
from tqdm import tqdm #for loop time
from hapi import *

HAPI version: 1.2.2.2
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2021 are now available in HAPI code

           MIT license: Copyright 2021 HITRAN team, see more at http://hitran.org. 

           If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005

           ATTENTION: This is the core version of the HITRAN Application Programming Interface.
                      For more efficient implementation of the absorption coefficient routine, 
                      as well as for new profiles, parameters and other functional,
      

## Data exporting for different data bases

In [2]:
data_hitran = read_hitran2012_parfile('65f81198.par')
data_hight = read_hight_file('bezard_hotco2_1mu_cdsd.dat')
data_ames = read_ames_file('Ames-2016.natural.co2.296k.list.long.format.dat')

Reading "65f81198.par" ...
Reading "bezard_hotco2_1mu_cdsd.dat" ...
Reading "Ames-2016.natural.co2.296k.list.long.format.dat" ...


### pressure, height, density, temperature profile of the atmoshpere with altitude 

In [3]:
pressure, height, density, temperature = np.loadtxt('VIRAPROFILE.txt',skiprows = 1, unpack=True)

## Doppler and Lorentz

In [5]:
Na = scipy.constants.N_A # mol^-1
ka  = 1.380649e-16 # erg K^−1
c = 2.99792458e10 # cms^−1
c2 = 1.4387769 # cm K

def Do(w0,T, Molm):
    
    wD = w0/c*np.sqrt(2*Na*ka*T/Molm) 

    return wD

def Lo(w0, p, T, gamma_self, p_self, pshift, n):
    
    wL = ((296/T)**n)*(gamma_self*p_self) #gamma_air*(p-p_self)+
    
    return wL

## Wavenumber range

In [6]:
#np.where((data_ames['linecenter'] >= 9400) & (data_ames['linecenter'] <= 11000))
#data_ames['linecenter'][1044063:1107731]

In [7]:
i = 1032907 #1044063
j = 1118650 #1107731
m = j-i
m

85743

## Data

In [8]:
E = np.array(data_ames['Epp'])[i:j]
sw = np.array(data_ames['S'])[i:j]
nu = np.array(data_ames['linecenter'])[i:j]
iso = np.array(data_ames['I'])[i:j]

gamma_self = np.array(data_ames['gamma-self'])[i:j]
pshift = np.array(data_ames['delta'])[i:j]
n = np.array(data_ames['N'])[i:j]
Molm = 43.989830

In [9]:
molar_mass = np.array([43.989830, 44.993185, 45.994076, 44.994045, 46.997431, 45.997400, 47.998320])
mol_mass= []
for j in np.arange(0,m):
    if iso[j] == 1:
        mol = molar_mass[0]
    elif iso[j] == 2:
        mol = molar_mass[1]
    else:
        mol = molar_mass[2]
    mol_mass.append(mol)

## Calculating cross-sections

In [10]:
w = np.arange(9150, 11250.01, 0.1) #wavenumber range for calculating profiles 9400-1000.01 cm^-1 with cut-off ±250 cm^-1

In [11]:
spectrum = []
#Loop for viraprofile of temperature and pressure
for l in tqdm(np.arange(0,101)):
    
    p_self = pressure[l]
    T = temperature[l]
    p = pressure[l]


    wD = Do(nu,T, mol_mass) #calculating half-width at half-maximum for doppler 
    wL = Lo(nu, p, T, gamma_self, p_self, pshift, n) #for Lorentz 
    
    #Calculating the spectrum of cross-sections
    sigma = np.zeros(w.size)
    for k in np.arange(0,m):
        #Calculating line strength
        Q = partitionSum(2,iso[k],296) #uses hapi module to calculate partition sums
        Qt = partitionSum(2,iso[k],T)
        intensities = sw[k]*Q/Qt*(np.exp(-c2*E[k]/T)/np.exp(-c2*E[k]/296))*(1 - np.exp(-c2*nu[k]/T))/(1 - np.exp(-c2*nu[k]/296))
        ## ======================================================

        nu[k] = nu[k] + pshift[k] *p 
        
        #chi facotor 
        delta_nu = np.absolute(w - nu[k])
    
        mask1 = delta_nu < 3 
        mask2 = (delta_nu > 3)&(delta_nu<20)
        mask3 = (delta_nu > 20)&(delta_nu<120)
        mask4 = (delta_nu>120)&(delta_nu<250)
        mask5 = delta_nu>250 #line cut-off

        chi1 = np.ones(delta_nu[mask1].shape)
    
        chi2 = 1.22*np.exp(-delta_nu[mask2]/15)
    
        chi3 = 0.3477*np.exp(-delta_nu[mask3]/260)
    
        chi4 = 0.7276*np.exp(-delta_nu[mask4]/100)

        chi5 = 0
        
        #Calculating voight profiles with Humlıcek approximation
 
        y = wL[k] / wD[k]
        x = (w-nu[k]) / wD[k]

        scale = 1/(wD[k]*np.sqrt(np.pi))
        
        profile = hum1zpf16m(x, y)*scale

        
        profile[mask1] = profile[mask1]*chi1
        profile[mask2] = profile[mask2]*chi2
        profile[mask3] = profile[mask3]*chi3
        profile[mask4] = profile[mask4]*chi4
        profile[mask5] = profile[mask5]*chi5
        
 
        # calculating cross_sections
    
        cross_section = profile*intensities
        cross_section = cross_section.real
        
    
        # calculating spectrum
        sigma += cross_section
        
    spectrum.append(sigma)

100%|███████████████████████████████████████| 101/101 [1:49:42<00:00, 65.18s/it]


## Calculating extinction

In [19]:
ex = density*1e5
ex_profiles = []
for z in tqdm(np.arange(0,101)):
    ep = spectrum[z]*ex[z]
    ex_profiles.append(ep)

100%|███████████████████████████████████████| 101/101 [00:00<00:00, 3863.21it/s]


In [20]:
ex_profiles = np.asarray(ex_profiles)
ex_profiles[:,2500:18501].shape

(101, 16001)

## Exporting to binary file

In [21]:
ex_profiles = np.asarray(ex_profiles)
ex_profiles_final = ex_profiles[:,2500:18501]  #selecting the right range of values 
rev = ex_profiles_final[::-1]
rev.shape

(101, 16001)

In [22]:
rev.tofile('EXTCLCO2_input_disort_venus10', format = '%02.0f') #exporting to binary file